### Load Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pandas pyarrow datasets

In [ ]:
from drive.MyDrive.Berkeley.MIDS.DATASCI_266.project.scripts.exact_match import ExactMatch

In [ ]:
data_file_path = "/content/drive/MyDrive/Berkeley/MIDS/DATASCI_266/project/data/nq_open/validation-00000-of-00001.parquet"

In [ ]:
import pandas as pd

# Load the Parquet file
df = pd.read_parquet(data_file_path, engine='pyarrow')

In [ ]:
df.head()

,question,answer
0,when was the last time anyone was on the moon,"[14 December 1972 UTC, December 1972]"
1,who wrote he ain't heavy he's my brother lyrics,"[Bobby Scott, Bob Russell]"
2,how many seasons of the bastard executioner ar...,"[one, one season]"
3,when did the eagles win last super bowl,[2017]
4,who won last year's ncaa women's basketball,[South Carolina]


### Load LLM

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
%pip install llama-index-vector-stores-pinecone
%pip install llama-index-embeddings-huggingface

In [ ]:
# sentence transformers
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
!pip install -q sentencepiece

In [ ]:
!pip install -q transformers

In [ ]:
!pip install -q accelerate

In [ ]:
from collections import Counter
import numpy as np
import tensorflow as tf
import accelerate
from tensorflow import keras

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [ ]:
flan_t5_base_tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
flan_t5_base_model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base", device_map="auto")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

## Build Pinecone Index

In [ ]:
from pinecone import Pinecone, ServerlessSpec
import os

api_key = '************************************'
pc = Pinecone(api_key=api_key)

In [ ]:
pinecone_index = pc.Index("paq-qa-pairs-bge-small-en")

## Vector store

In [ ]:
from llama_index.vector_stores.pinecone import PineconeVectorStore

In [ ]:
vector_store = PineconeVectorStore(pinecone_index=pinecone_index)

In [ ]:
SIMILARITY_TOP_K = 10

## Inference

### Sample prompt template and context with RAG

In [ ]:
PROMPT_TEMPLATE = """
Given the below context:\n{context}\n\nAnswer the following \n{question}\n
"""

CONTEXT_TEMPLATE = """
The context is a list of questions and answers pairs that could be used to answer the questions.
\"\"\"
{context}
\"\"\"
"""

context = """
The context is a list of questions and answers pairs that could be used to answer the questions.
\"\"\"
{"question":"when was the last time philadelphia eagles won a superbowl","answer":["2017"]}
{"question":"when was the last time the eagles won the superbowl","answer":["2017"]}
{"question":"when was the last time the eagles won a superbowl","answer":["2017"]}
{"question":"when did the philadelphia eagles win their first super bowl","answer":["1947"]}
{"question":"when was the last time the eagles went to the superbowl","answer":["2017"]}
{"question":"when was the last time the eagles beat the steelers","answer":["2016"]}
{"question":"when is the last time the eagles beat the redskins","answer":["December 11, 2016"]}
{"question":"who did the eagles play in the super bowl","answer":["New England Patriots"]}
{"question":"who did the eagles play in the 2004 super bowl","answer":["New England Patriots"]}
{"question":"who did the philadelphia eagles play in the 2004 super bowl","answer":["New England Patriots"]}
{"question":"who did the eagles beat in the super bowl","answer":["New England Patriots"]}
{"question":"when was the last time the eagles were in the playoffs","answer":["2017"]}
{"question":"when was the last time the eagles played the steelers","answer":["2016"]}
{"question":"when did the eagles win the nfc east","answer":["1988"]}
{"question":"when did the atlanta falcons win their first super bowl","answer":["1998"]}
{"question":"when was the last time the falcons were in the super bowl","answer":["2016"]}
{"question":"when did emmitt smith win super bowl","answer":["1993"]}
{"question":"when did the eagles win the nfl championship","answer":["1948"]}
{"question":"who did philadelphia eagles play in the super bowl","answer":["New England Patriots"]}
{"question":"when did the eagles win the miracle","answer":["December 19, 2010"]}
\"\"\"
"""

In [ ]:
input_text = PROMPT_TEMPLATE.format(context = context, question='when did the eagles win last super bowl')
print(input_text)
input_ids = flan_t5_base_tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

outputs = flan_t5_base_model.generate(input_ids)
print(flan_t5_base_tokenizer.decode(outputs[0]))

Token indices sequence length is longer than the specified maximum sequence length for this model (777 > 512). Running this sequence through the model will result in indexing errors



Given the below context:

The context is a list of questions and answers pairs that could be used to answer the questions.
"""
{"question":"when was the last time philadelphia eagles won a superbowl","answer":["2017"]}
{"question":"when was the last time the eagles won the superbowl","answer":["2017"]}
{"question":"when was the last time the eagles won a superbowl","answer":["2017"]}
{"question":"when did the philadelphia eagles win their first super bowl","answer":["1947"]}
{"question":"when was the last time the eagles went to the superbowl","answer":["2017"]}
{"question":"when was the last time the eagles beat the steelers","answer":["2016"]}
{"question":"when is the last time the eagles beat the redskins","answer":["December 11, 2016"]}
{"question":"who did the eagles play in the super bowl","answer":["New England Patriots"]}
{"question":"who did the eagles play in the 2004 super bowl","answer":["New England Patriots"]}
{"question":"who did the philadelphia eagles play in the 2004

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


<pad> When did the eagles win the miracle?</s>


In [ ]:
from llama_index.core.vector_stores import VectorStoreQuery

query_mode = "default"

def generate_answer(question, index):
    try:
        # question embedding
        question_embedding = embed_model.get_query_embedding(question)

        # construct vector store query
        vector_store_query = VectorStoreQuery(
          query_embedding=question_embedding, similarity_top_k=SIMILARITY_TOP_K, mode=query_mode
        )

        # returns a VectorStoreQueryResult
        query_result = vector_store.query(vector_store_query)
        query_results_nodes = [node.get_content() for node in query_result.nodes]

        # Concatenate the list of strings into a single string separated by '\n'
        context = '\n'.join(query_results_nodes)

        # Assuming PROMPT_TEMPLATE is defined elsewhere and uses {context} and {question}
        input_text = PROMPT_TEMPLATE.format(context=context, question=question)

        # Assuming flan_t5_large_tokenizer and flan_t5_large_model are defined and initialized elsewhere
        input_ids = flan_t5_base_tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")
        outputs = flan_t5_base_model.generate(input_ids)
        answer = flan_t5_base_tokenizer.decode(outputs[0])

        print(f"Index: {index}, Question: '{question}', Generated Answer: '{answer}'")
        return answer
    except Exception as e:
        print(f"Error generating answer for question '{question}' at index {index}: {e}")
        return None

In [ ]:
print(generate_answer("when did the eagles win last super bowl", 0))

Index: 0, Question: 'when did the eagles win last super bowl', Generated Answer: '<pad> 2007</s>'
<pad> 2007</s>


In [ ]:
eval_file_path = "/content/drive/MyDrive/Berkeley/MIDS/DATASCI_266/project/eval/nq_open_flan-t5-base+RAG.csv"

In [ ]:
try:
    # Try to load the DataFrame from the CSV file
    df_eval = pd.read_csv(eval_file_path)
    print("Data loaded from file.")
except FileNotFoundError:
    # If the file doesn't exist, copy df into df_eval
    df_eval = df.copy()
    if 'predictions' not in df_eval.columns:
      df_eval['predictions'] = pd.NA  # or use None or np.nan, depending on your preference
    print("File not found. Copied df into df_eval.")

File not found. Copied df into df_eval.


In [ ]:
from tqdm import tqdm

# Initialize a counter for tracking when to save the DataFrame
save_counter = 0

for index, row in tqdm(df_eval.iterrows(), total=df_eval.shape[0], desc="Generating Answers"):
    # Check if 'predictions' is empty for the current row
    if pd.isna(row['predictions']) or row['predictions'] == '':
        # Generate the answer only if 'predictions' is empty
        df_eval.at[index, 'predictions'] = generate_answer(row['question'], index)

        # Increment the save counter
        save_counter += 1

        # Save the DataFrame back to the file every 10 rows
        if save_counter % 10 == 0:
            df_eval.to_csv(eval_file_path, index=False)
            print(f"Progress saved at index {index}")

# Save any remaining progress at the end of the loop
if save_counter % 10 != 0:
    df_eval.to_csv(eval_file_path, index=False)
    print("Final progress saved.")

Generating Answers:   0%|          | 0/3610 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Generating Answers:   0%|          | 1/3610 [00:00<43:02,  1.40it/s]

Index: 0, Question: 'when was the last time anyone was on the moon', Generated Answer: '<pad> 14 November 2016</s>'


Generating Answers:   0%|          | 2/3610 [00:01<29:06,  2.07it/s]

Index: 1, Question: 'who wrote he ain't heavy he's my brother lyrics', Generated Answer: '<pad> Bobby Scott</s>'


Generating Answers:   0%|          | 3/3610 [00:01<34:24,  1.75it/s]

...

Index: 3607, Question: 'who will take the throne after the queen dies', Generated Answer: '<pad> William III</s>'


Generating Answers: 100%|█████████▉| 3609/3610 [16:00<00:00,  3.60it/s]

Index: 3608, Question: 'what domain has more individuals than all other domains combined do', Generated Answer: '<pad>.coop</s>'


Generating Answers: 100%|██████████| 3610/3610 [16:00<00:00,  3.76it/s]

Index: 3609, Question: 'what is the meaning of the name comanche', Generated Answer: '<pad> enemy, stranger</s>'
Progress saved at index 3609


In [ ]:
df_eval.head()

,question,answer,predictions
0,when was the last time anyone was on the moon,"[14 December 1972 UTC, December 1972]",<pad> 14 November 2016</s>
1,who wrote he ain't heavy he's my brother lyrics,"[Bobby Scott, Bob Russell]",<pad> Bobby Scott</s>
2,how many seasons of the bastard executioner ar...,"[one, one season]",<pad> two</s>
3,when did the eagles win last super bowl,[2017],<pad> 2007</s>
4,who won last year's ncaa women's basketball,[South Carolina],<pad> Tennessee</s>


## Save evaluation data

In [ ]:
df_eval.to_csv(eval_file_path, index=False)

## Evaluation metrics

In [ ]:
!pip install pandas pyarrow datasets

In [ ]:
import pandas as pd

In [ ]:
eval_file_path = "/content/drive/MyDrive/Berkeley/MIDS/DATASCI_266/project/eval/nq_open_flan-t5-base+RAG.csv"

In [ ]:
df = pd.read_csv(eval_file_path)

In [ ]:
df.head()

,question,answer,predictions
0,when was the last time anyone was on the moon,['14 December 1972 UTC' 'December 1972'],<pad> 14 November 2016</s>
1,who wrote he ain't heavy he's my brother lyrics,['Bobby Scott' 'Bob Russell'],<pad> Bobby Scott</s>
2,how many seasons of the bastard executioner ar...,['one' 'one season'],<pad> two</s>
3,when did the eagles win last super bowl,['2017'],<pad> 2007</s>
4,who won last year's ncaa women's basketball,['South Carolina'],<pad> Tennessee</s>


In [ ]:
print(df.iloc[1]['answer'])

['Bobby Scott' 'Bob Russell']


In [ ]:
import ast
import string
import re
from drive.MyDrive.Berkeley.MIDS.DATASCI_266.project.scripts.exact_match import normalize_answer

def string_to_list(string):
    try:
        # Convert the string to a list
        matches = re.findall(r"'(.*?)'", string)
        return matches
    except Exception as e:
        # In case of an error, print the error and return the original string
        print(f"Error converting string to list: {e}")
        return string


def preprocess(text: str, answers: list) -> str:
    tokens_to_remove = ["<pad> ", "</s>"]

    # Remove specific tokens
    for token in tokens_to_remove:
        text = text.replace(token, '')

    exclude = set(string.punctuation)
    exclude.add("’")
    exclude.add("\n")

    # Remove excluded characters
    cleaned_text = normalize_answer("".join(ch for ch in text if ch not in exclude))

    # Replace text with answer item if it matches a substring in the cleaned text
    for answer_item in answers:
        if normalize_answer(answer_item) in cleaned_text:
            return answer_item  # Return the answer item that matches the substring

    return cleaned_text

In [ ]:
df['answer'] = df['answer'].apply(string_to_list)
df['predictions'] = df['predictions'].str.replace("<pad> ", "")
df['predictions'] = df['predictions'].str.replace("</s>", "")
df['predictions_em'] = df.apply(lambda row: preprocess(row['predictions'], row['answer']), axis=1)

In [ ]:
df.head()

,question,answer,predictions,predictions_em
0,when was the last time anyone was on the moon,"[14 December 1972 UTC, December 1972]",14 November 2016,14 november 2016
1,who wrote he ain't heavy he's my brother lyrics,"[Bobby Scott, Bob Russell]",Bobby Scott,Bobby Scott
2,how many seasons of the bastard executioner ar...,"[one, one season]",two,two
3,when did the eagles win last super bowl,[2017],2007,2007
4,who won last year's ncaa women's basketball,[South Carolina],Tennessee,tennessee


### Calculate Scores

In [ ]:
import datasets
from drive.MyDrive.Berkeley.MIDS.DATASCI_266.project.scripts.exact_match import exact_match_score, metric_max_over_ground_truths, f1_score

df['em'] = df.apply(lambda row: metric_max_over_ground_truths(exact_match_score, row['predictions_em'], row['answer']) if row['answer'] and row['predictions_em'] else 0, axis=1)
df['f1'] = df.apply(lambda row: metric_max_over_ground_truths(f1_score, row['predictions_em'], row['answer']) if row['answer'] and row['predictions_em'] else 0, axis=1)

In [ ]:
!pip install -q rouge_score
!pip install -q evaluate

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.7 MB/s eta 0:00:00


In [ ]:
import evaluate

df['predictions_em'] = df['predictions_em'].astype(str)
df['answer'] = df['answer'].astype(str)

# Filter out rows where either 'predictions' or 'answer' is empty or NaN
filtered_df = df[(df['predictions_em'].notna() & df['predictions_em'].str.strip().astype(bool)) &
                 (df['answer'].notna() & df['answer'].str.strip().astype(bool))]
rouge = evaluate.load('rouge')
predictions = filtered_df['predictions_em'].tolist()
references = filtered_df['answer'].tolist()
rouge_results = rouge.compute(predictions=predictions,
                        references=references)
print(rouge_results)

{'rouge1': 0.24648320728402767, 'rouge2': 0.12897899623949632, 'rougeL': 0.24605934492566564, 'rougeLsum': 0.2458691141142707}


### Prove of correctness

#### Top 10 rows from data frame

In [ ]:
df.head(10)

,question,answer,predictions,predictions_em,em,f1
0,when was the last time anyone was on the moon,"['14 December 1972 UTC', 'December 1972']",14 November 2016,14 november 2016,False,0.285714
1,who wrote he ain't heavy he's my brother lyrics,"['Bobby Scott', 'Bob Russell']",Bobby Scott,Bobby Scott,True,1.000000
2,how many seasons of the bastard executioner ar...,"['one', 'one season']",two,two,False,0.000000
3,when did the eagles win last super bowl,['2017'],2007,2007,False,0.000000
4,who won last year's ncaa women's basketball,['South Carolina'],Tennessee,tennessee,False,0.000000
5,when did the isle of wight become an island,['During the last Ice Age'],spring 1991,spring 1991,False,0.000000
6,love yourself by justin bieber is about who,['Rihanna'],a girl who is a sex addict,girl who is sex addict,False,0.000000
7,who was the ruler of england in 1616,['James I'],Edward VII,edward vii,False,0.000000
8,what is the hot coffee mod in san andreas,['a normally inaccessible mini-game'],Patch,patch,False,0.000000
9,what is the maximum data rate for the 802.11a ...,['54\\xa0Mbit/s'],2.4 GHz,24 ghz,False,0.000000


#### Top 10 rows from data frame that exactly match one of the reference answer

In [ ]:
df_em_true = df[df['em'] == True]
df_em_true.head(10)

,question,answer,predictions,predictions_em,em,f1
1,who wrote he ain't heavy he's my brother lyrics,"['Bobby Scott', 'Bob Russell']",Bobby Scott,Bobby Scott,True,1.0
11,who sang i ran all the way home,['The Impalas'],The Impalas,The Impalas,True,1.0
14,who does the voice of nala in the lion king,"['Niketa Calame', 'Sally Dworsky', 'Moira Kell...",Moira Kelly,Moira Kelly,True,1.0
19,what age do you need to be to buy a bb gun,['18'],18,18,True,1.0
23,how many episodes are there in dragon ball z,"['291 episodes', '291']",291,291,True,1.0
24,who plays auggie in the movie the wonder,['Jacob Tremblay'],Jacob Tremblay,Jacob Tremblay,True,1.0
28,who plays joker in batman the dark knight,['Ledger'],Heath Ledger,Ledger,True,1.0
30,when was the first hunger games book published,"['September 14, 2008', '2008']",2008,2008,True,1.0
33,where did the butchers in the slaughterhouse c...,['New Orleans'],New Orleans,New Orleans,True,1.0
37,when did taylor swift's first album release,"['October 24, 2006', '2005']","October 24, 2006","October 24, 2006",True,1.0


#### Top 10 rows from data frame that doesn't match match any of the reference answer

In [ ]:
df_em_true = df[df['em'] == False]
df_em_true.head(10)

,question,answer,predictions,predictions_em,em,f1
0,when was the last time anyone was on the moon,"['14 December 1972 UTC', 'December 1972']",14 November 2016,14 november 2016,False,0.285714
2,how many seasons of the bastard executioner ar...,"['one', 'one season']",two,two,False,0.000000
3,when did the eagles win last super bowl,['2017'],2007,2007,False,0.000000
4,who won last year's ncaa women's basketball,['South Carolina'],Tennessee,tennessee,False,0.000000
5,when did the isle of wight become an island,['During the last Ice Age'],spring 1991,spring 1991,False,0.000000
6,love yourself by justin bieber is about who,['Rihanna'],a girl who is a sex addict,girl who is sex addict,False,0.000000
7,who was the ruler of england in 1616,['James I'],Edward VII,edward vii,False,0.000000
8,what is the hot coffee mod in san andreas,['a normally inaccessible mini-game'],Patch,patch,False,0.000000
9,what is the maximum data rate for the 802.11a ...,['54\\xa0Mbit/s'],2.4 GHz,24 ghz,False,0.000000
10,which state is located in the centre of india,"['Chhattisgarh', 'Madhya Pradesh']",Uttar Pradesh,uttar pradesh,False,0.500000


#### Overall Exact Match & F1 & Rouge scores

In [ ]:
overall_em = sum(df['em']) / len(df['em'])
overall_f1 = sum(df['f1']) / len(df['f1'])

# Print the overall Exact Match score, F1 and Rouge score
print(f"Overall Exact Match Score: {overall_em * 100:.2f}")
print(f"Overall F1 Score: {overall_f1 * 100:.2f}")
print(f"Rouge1: {rouge_results['rouge1'] * 100:.2f}")
print(f"Rouge2: {rouge_results['rouge2'] * 100:.2f}")
print(f"RougeL: {rouge_results['rougeL'] * 100:.2f}")
print(f"RougeLSum: {rouge_results['rougeLsum'] * 100:.2f}")

Overall Exact Match Score: 27.59
Overall F1 Score: 32.27
Rouge1: 24.65
Rouge2: 12.90
RougeL: 24.61
RougeLSum: 24.59


#### Save with metrics

In [ ]:
eval_with_metrics_file_path = "/content/drive/MyDrive/Berkeley/MIDS/DATASCI_266/project/eval/nq_open_flan-t5-base+RAG_metrics.csv"

In [ ]:
df.to_csv(eval_with_metrics_file_path, index=False)